# PROJET：From knowledge extraction to knowledge representation
## Extract knowledge from text using Naîve Bayes and SVM

## Use google colab to upload the files into google driver

from google.colab import files

!mkdir text

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
       name=fn, length=len(uploaded[fn])))

!mv *.txt ./text
!ls ./text

## Load training set and test set
__ Data Sources: fetch_20newsgroups

In [84]:
#Loading the data set - training data.
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
twenty_test = fetch_20newsgroups(subset='test')

# Removing stop words with NLTK

In [85]:
import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')
#word_tokenize accepts a string as an input, not a file. 
def remove_stopwords(word_list):
        processed_word_list = []
        for word in word_list:
            word = word.lower() # in case they arenet all lower cased
            if word not in stopwords.words("english"):
                processed_word_list.append(word)
        return processed_word_list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\William\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## View 30 news in the system path

In [86]:
#打开所有TXT文档
import os
import codecs
import io
filepath = "./NEWS_AI/NEWS"
text_name = os.listdir("./NEWS_AI/NEWS")
article_30 = []
title_30 = []
document_30 = []
news_30 = []
datas = set()
i = 0
for s in text_name:
    newDir=os.path.join(filepath,s)     #将文件命加入到当前文件路径后面
    if os.path.isfile(newDir) :         #如果是文件
        if os.path.splitext(newDir)[1]==".txt":  #判断是否是txt
            i = i+1
            f = codecs.open(newDir,encoding='utf-8', errors='ignore').read()
            news_30.append(f)
            words = f.split()
            article_30.append(" ".join(remove_stopwords(words)))
            title_30.append(os.path.splitext(newDir)[0].replace("./NEWS_AI/NEWS\\",""))
            document_30.append("document"+str(i))
            datas.add(f)
        pass
    else:
        eachFile(newDir)                #如果不是文件，递归这个文件夹的路径


In [87]:
title_30[1]

'Around the NBA skepticism over whether Wizards would actually trade John Wall or Bradley Beal'

In [88]:
document_30[1]

'document2'

In [89]:
news_30[1]

"Around the NBA, skepticism over whether Wizards would actually trade John Wall or Bradley Beal\r\n \r\nCould the Wizards' backcourt be headed toward a breakup? Other teams don't seem to think so. (John McDonnell/The Washington Post)\r\nBy Candace Buckner November 19 at 5:22 PM\r\nBradley Beal checks his phone just like anyone else. He has read the rumors about the Washington Wizards and their sudden inclination to discuss potential trades involving Beal or fellow all-star guard John Wall.\r\n\r\nFollowing the Wizards' latest indifferent performance that resulted in a loss, ESPNs Adrian Wojnarowski reported Monday that Washingtons management has signaled to other teams an impression that every player on their roster could be available for trade discussions.\r\n\r\nIm not going to be naive to it, you know, said Beal, who saw the report. Ive heard those rumors weeks ago. Then, I didnt buy into them. Now, Im still not going to buy into them because if thats my main priority and focus, the

In [90]:
article_30[1]

"around nba, skepticism whether wizards would actually trade john wall bradley beal could wizards' backcourt headed toward breakup? teams seem think so. (john mcdonnell/the washington post) candace buckner november 19 5:22 pm bradley beal checks phone like anyone else. read rumors washington wizards sudden inclination discuss potential trades involving beal fellow all-star guard john wall. following wizards' latest indifferent performance resulted loss, espns adrian wojnarowski reported monday washingtons management signaled teams impression every player roster could available trade discussions. im going naive it, know, said beal, saw report. ive heard rumors weeks ago. then, didnt buy them. now, im still going buy thats main priority focus, im going messed floor. beals disbelief apparently warranted. although washington may floundering 5-11 start season, belief team plans move either franchise cornerstones, based conversations multiple people league familiarity wizards front office op

# Print the categories names and some text examples

In [91]:
# You can check the target names (categories) by using target_names
# the texts are stored in the data attribute of the variable twinty train
twenty_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [92]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english')
X_test_counts = count_vect.fit_transform(twenty_test.data)
X_test_counts.shape

(7532, 93255)

# Features extraction from text data

In [93]:
#check the scikit documentation to see definition of CoutVectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(11314, 129796)

In [94]:
# TF-IDF is an other text preprocessing procedure : Check its definition and apply it using the following code
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Try a machine learning algorithm

In [95]:
# Training Naive Bayes (NB) classifier on training data.
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

# Build a pipeline including preprocessings and learning steps

In [96]:
# Building a pipeline: We can write less code and do all of the above, by building a pipeline as follows:
# We will be using the 'text_clf' going forward.
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

# Evaluate your model performance

Exercice : evaluate the performance of the first model (named clf) on the 20 newsgroups test data 

In [97]:
docs_test = twenty_test.data

In [98]:
clf_res = text_clf.predict(docs_test)

In [99]:
import numpy as np
print("ACC：")
print(np.mean(clf_res == twenty_test.target))

ACC：
0.8169144981412639


# Evaluate MY news_30 model performance

In [100]:
new_predict = text_clf.predict(news_30)

In [101]:
new_predict

array([14, 10, 10, 14, 14, 18, 18, 17, 17, 14, 10, 17, 10, 10, 10, 13, 16,
       14, 14, 14, 18, 15, 15, 17, 14, 17, 16, 17,  9, 12])

# Exercice : Using scikit pipeline and SVM Classifier (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html), create a model based on the 20newsgroup dataset and evaluate its performance.

In [102]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge',
                                            penalty='l2',
                                            alpha=1e-3,
                                            n_iter=5,
                                            random_state=42)),
                    ])
text_clf_svm = text_clf_svm.fit(twenty_train.data, twenty_train.target)
predicted = text_clf_svm.predict(docs_test)
print("ACC：")
print(np.mean(predicted == twenty_test.target))

F:\Anaconda\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


ACC：
0.8224907063197026


In [103]:
new_predict_svm = text_clf_svm.predict(news_30)

In [104]:
new_predict_svm

array([14, 10, 10, 14, 14, 18, 18, 17, 18, 14, 10, 18, 10, 10, 14, 10, 17,
       14, 14, 14, 18,  7, 13, 18, 14, 17, 13, 17,  9, 12])

# Grid Search : Trying different hyperparameters

In [105]:
# First, we are creating a list of parameters for which we would like to do performance tuning. 
# All the parameters name start with the classifier name.
# E.g. vect__ngram_range; here we are telling to use unigram and bigrams and choose the one which is optimal.

from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

In [106]:
# Next, we create an instance of the grid search by passing the classifier, parameters 
# and n_jobs=-1 which tells to use multiple cores from user machine.

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

In [107]:
# To see the best mean score and the params, run the following code

print(gs_clf.best_score_)
print(gs_clf.best_params_)

# Output for above should be: The accuracy has now increased to ~90.6% for the NB classifier
# and the corresponding parameters are {‘clf__alpha’: 0.01, ‘tfidf__use_idf’: True, ‘vect__ngram_range’: (1, 2)}.

0.9057804490012374
{'clf__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}


In [108]:
gs_clf.predict(news_30)

array([14, 10, 10, 14, 14, 18, 18, 17, 18, 16, 10, 18, 10, 10, 16, 13, 16,
       14, 14, 14, 18, 14, 13, 18, 14, 17, 18, 17,  9, 12])

# Grid Search for SVM : Similarly, do grid search for SVM and output the best score and parameters

In [109]:
# 查看分类器对于新文本的预测结果，你可以自行改变下方的字符串来观察分类效果
gs_clf_svm = GridSearchCV(text_clf_svm, parameters, n_jobs=-1)
gs_clf_svm = gs_clf.fit(twenty_train.data, twenty_train.target)

In [110]:
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

0.9057804490012374
{'clf__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}


In [111]:
gs_clf_svm.predict(news_30)

array([14, 10, 10, 14, 14, 18, 18, 17, 18, 16, 10, 18, 10, 10, 16, 13, 16,
       14, 14, 14, 18, 14, 13, 18, 14, 17, 18, 17,  9, 12])

#save as CSV


'alt.atheism',

 'comp.graphics',
 
 'comp.os.ms-windows.misc',
 
 'comp.sys.ibm.pc.hardware',
 
 'comp.sys.mac.hardware',
 
 'comp.windows.x',
 
 'misc.forsale',
 
 'rec.autos',
 
 'rec.motorcycles',
 
 'rec.sport.baseball',
 
 'rec.sport.hockey',
 
 'sci.crypt',
 
 'sci.electronics',
 
 'sci.med',
 
 'sci.space',
 
 'soc.religion.christian',
 
 'talk.politics.guns',
 
 'talk.politics.mideast',
 
 'talk.politics.misc',
 
 'talk.religion.misc'

In [112]:
import pandas as pd
news_result = pd.DataFrame(document_30,columns=["document"])
news_result["title"] = title_30
news_result["subject"] = gs_clf_svm.predict(news_30)
news_result = news_result.replace({7:"misc.forsale",8:"rec.autos",9:"rec.motorcycles",10:"rec.sport.baseball",11:"rec.sport.hockey",12:"sci.crypt",13:"sci.electronics",14:"sci.med",15:"sci.space",16:"soc.religion.christian",17:"talk.politics.guns",18:"talk.politics.mideast",19:"talk.politics.misc",20:"talk.religion.misc"})

In [113]:
news_result.to_csv ("text_news.csv" , encoding = "utf-8")

In [114]:
# from google.colab import files


# files.download('text_news.csv')